In [1]:
import json
import csv 
import pandas as pd

from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, FOAF, XSD

In [2]:
data_business = []
csvfile = []
cuisineList = []
existdbList = []
specialList = []

In [3]:
with open('yelp_academic_dataset_business.json', encoding="utf8") as f:
    for line in f:
        data_business.append(json.loads(line))
df = pd.read_csv('csvData.csv')

with open('country_cusine_filtered_list.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        cuisineList.append(row[0])
        
with open('category_exist_in_db_List.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        existdbList.append(row[0])
        
with open('distinct_with_specialchar.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        specialList.append(row[0])

In [4]:
exist1 = pd.read_csv('category_exist_in_db_List.csv',header=None)
exist2 = pd.read_csv('category_exist_in_db_List_v2.csv', header=None)
exist3 = pd.read_csv('category_exist_in_db_List_v3.csv', header=None)

frames = [exist1, exist2, exist3]
result = pd.concat(frames)
result = result.reset_index(drop=True)
#exist_v = exist2.add(exist3, fill_value=0)

display(result)

,0,1
0,Doctors,NaN
1,Acupuncture,NaN
2,Nutritionists,NaN
3,Notaries,NaN
4,Shopping,NaN
...,...,...
770,Gerontologists,Gerontologist
771,Otologists,Otologist
772,Parklets,Parklet
773,Neurotologists,Neurotologist


In [5]:
def string_seperate(string):
    n = 0
    cate_list = []
    a_list = string.split(", ")
    for element in a_list:
        cate_list.append(element)
    n = n +1
    return cate_list

def string_seperate_v2(string):
    n = 0
    cate_list = []
    a_list = string.replace('{', '')
    a_list = a_list.replace('}', '')
    a_list = a_list.split(", ")
    for element in a_list:
        cate_list.append(element)
    n = n +1
    return cate_list

def code_to_state(code):
    n = 0
    for index, row in df.iterrows():
        if str(code) == str(df.loc[n,'Code']):
            return str(df.loc[n,'State'])
        #str(df.loc[n,])
        n = n + 1
        

def change_format_to_db(catagory):
    #this function takes a string in catagory from yelp
    #and gives the correct format to put in dbpedia
    
    #Everthing in string to lowercase
    newcatagory = catagory.lower()
    
    #capitalize first letter 
    newcatagory = catagory.capitalize()
    
    #replace empty with _
    newcatagory = catagory.replace(" ", "_")
    
    return newcatagory

In [6]:
schema = Namespace("https://schema.org/")
yelp_business = Namespace("https://www.yelp.com/biz/")
db_resource = Namespace("http://dbpedia.org/resource/")
yelp_ambience = Namespace("https://www.yelp.com/kg/biz/ambience/")
yelp_goodformeal = Namespace("https://www.yelp.com/kg/biz/ambience/goodformeal")
yelp_bestnights = Namespace("https://www.yelp.com/kg/biz/ambience/bestnights")
g_maps = geonames = Namespace("http://www.geonames.org/")

g = Graph()

In [7]:
def triple_maker_name():
    #for each name that exist for a business, add a triple to the graph
    n = 0
    for i in data_business:
        if len(data_business[n]['name']) > 0 and data_business[n]['name'] != None:
            #append to graph
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.legalName, Literal(data_business[n]['name'], datatype=XSD.string)))
        n = n + 1
    print('name done')

In [8]:
def triple_maker_address():
    #for each address that exist for a business, add a triple to the graph
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['address']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.streetAddress, Literal(data_business[n]['address'], datatype=XSD.string)))
        n = n + 1
    print('address done')

In [9]:
def triple_maker_state():
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['state']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.addressRegion, URIRef(db_resource + str(code_to_state(data_business[n]['state'])))))
        n = n + 1
        
    print('state done')

In [10]:
def triple_maker_postal():
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['postal_code']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.postalCode, Literal(data_business[n]['postal_code'], datatype=XSD.string)))
        n = n + 1
        
    print('postal done')

In [11]:
def triple_maker_latlong():
    n = 0
    for i in data_business:
        #append to graph
        if data_business[n]['latitude'] != None and len(str(data_business[n]['longitude'])) != None :
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.geo, URIRef(g_maps + str(data_business[n]['latitude']))))
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.geo, URIRef(g_maps + str(data_business[n]['longitude']))))
        n = n + 1
    print('lat+long done')

In [12]:
def triple_maker_stars():
    n = 0
    for i in data_business:
        #append to graph
        if len(str(data_business[n]['stars'])) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.starRating, Literal(data_business[n]['stars'])))
        n = n + 1
    print('stars done')

In [13]:
def triple_maker_cg_cuisine():
    n = 0
    for i in data_business:
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
                
            #for every item in catagory_list, compare it with the cuisineList
            for item in range(len(cuisineList)):
                for element in range(len(catagory_list)):
                    #the item in category exist, add it to the graph
                    if catagory_list[element] == cuisineList[item]:
                        #cause the cuisines exist as country+_cuisine in dbpedia
                        #so every country get added a _cuisine after
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_resource + str(cuisineList[element])+ str('_cuisine'))))
        n = n + 1
    print('category for cuisine done')  

In [14]:
def triple_maker_cg_in_db_exist():
                  
    n = n + 1
    for i in data_business:
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
                
            #for every item in catagory_list, compare it with a elemenent that exist in dbpedia
            item1 = 0
            for item in range(len(catagory_list)):
                element1 = 0
                for element in range(len(existdbList)):
                    #the item in category exist, add it to the graph
                    if str(catagory_list[item]) == str(existdbList[element]):
                        #the format in dbpedia is first letter is upper case
                        #and the rest of the letters are lowercase
                        #so every string goes through the function change_format_to_db
                        #in order to be the correct format
                        
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_resource + change_format_to_db(existdbList[element]))))
    print('category that exist in db done')

In [15]:
g2 = Graph()

def triple_maker_cg_in_db_exist_2():
    category_exist = 0    
    category_not_exist = 0  
    
    for n in range(len(data_business)):
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
            #for every item in catagory_list, compare it with the dbpedia
            item1 = 0
            for item in range(len(catagory_list)):
                element1 = 0
                for index, row in df.iterrows():
                    #the item in category exist, add it to the graph
                    if str(catagory_list[item]) == str(row[0]):
                        
                        #the format in dbpedia is first letter is upper case
                        #and the rest of the letters are lowercase
                        #so every string goes through the function change_format_to_db
                        #in order to be the correct format
                        #g2.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_resource + change_format_to_db(existdbList[element]))))
                        category_exist = category_exist + 1
                    else:
                        category_not_exist = category_not_exist + 1
    print('category_2 that exist in db done')

In [16]:
def triple_maker_att_RestaurantsTakeOut():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsTakeOut'):
                    if (data_business[n]['attributes']['RestaurantsTakeOut']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.keywords, URIRef(db_resource + 'Take-out')))

In [17]:
def triple_maker_att_BusinessParking():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BusinessParking'):
                    s_v2_list = string_seperate_v2(data_business[n]['attributes']['BusinessParking'])
                    for words in range(len(s_v2_list)):
                        if (s_v2_list[words]) == "'lot': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, URIRef(db_resource + 'Parking_lot', datatype=XSD.string)))
                        elif (s_v2_list[words]) == "u'lot': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, URIRef(db_resource + 'Parking_lot', datatype=XSD.string)))
                        elif (s_v2_list[words]) == "'street': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, URIRef(db_resource + 'Street', datatype=XSD.string)))
                        elif (s_v2_list[words]) == "u'street': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, URIRef(db_resource + 'Street', datatype=XSD.string)))
                        elif (s_v2_list[words]) == "'validated': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, Literal('Validated', datatype=XSD.string)))
                        elif (s_v2_list[words]) == "u'validated': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, Literal('Validated', datatype=XSD.string)))
                        elif (s_v2_list[words]) == "'valet': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, Literal('Valet', datatype=XSD.string)))
                        elif (s_v2_list[words]) == "u'valet': True":
                            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, Literal('Valet', datatype=XSD.string)))   

In [18]:
def triple_maker_att_Smoking():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Smoking'):
                    if (data_business[n]['attributes']['Smoking']) == "'yes'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.smokingAllowed, Literal('True', datatype=XSD.boolean)))
                    elif (data_business[n]['attributes']['Smoking']) == "u'yes'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.smokingAllowed, Literal('True', datatype=XSD.boolean)))
                    elif (data_business[n]['attributes']['Smoking']) == "'no'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.smokingAllowed, Literal('False', datatype=XSD.boolean)))
                    elif (data_business[n]['attributes']['Smoking']) == "u'no'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.smokingAllowed, Literal('False', datatype=XSD.boolean)))

In [19]:
def triple_maker_att_BusinessAcceptsCreditCards():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BusinessAcceptsCreditCards'):
                    if (data_business[n]['attributes']['BusinessAcceptsCreditCards']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.paymentAccepted, Literal('Credit Card', datatype=XSD.string)))

In [20]:
def triple_maker_att_BikeParking():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BikeParking'):
                    if (data_business[n]['attributes']['BikeParking']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.ParkingFacility, URIRef(db_resource + 'Bicycle')))

In [21]:
def triple_maker_att_RestaurantsPriceRange2():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsPriceRange2'):
                    if (data_business[n]['attributes']['RestaurantsPriceRange2']) == '1':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('1')))
                    elif (data_business[n]['attributes']['RestaurantsPriceRange2']) == '2':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('2')))
                    elif (data_business[n]['attributes']['RestaurantsPriceRange2']) == '3':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('3')))
                    elif (data_business[n]['attributes']['RestaurantsPriceRange2']) == '4':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.priceRange, Literal('4')))

In [22]:
def triple_maker_att_CoatCheck():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('CoatCheck'):
                    if (data_business[n]['attributes']['CoatCheck']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Wardrobe')))

In [23]:
def triple_maker_att_RestaurantsDelivery():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsDelivery'):
                    if (data_business[n]['attributes']['RestaurantsDelivery']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.DeliveryMethod, URIRef(db_resource + 'Delivery_service')))

In [24]:
def triple_maker_att_Caters():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Caters'):
                    if (data_business[n]['attributes']['Caters']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.FoodEstablishment, URIRef(db_resource + 'Catering')))

In [25]:
def triple_maker_att_WiFi():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('WiFi'):
                    if (data_business[n]['attributes']['WiFi']) == "'free'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Wi-Fi')))
                    if (data_business[n]['attributes']['WiFi']) == "u'free'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Wi-Fi')))
                    if (data_business[n]['attributes']['WiFi']) == "'paid'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Wi-Fi')))
                    if (data_business[n]['attributes']['WiFi']) == "u'paid'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Wi-Fi')))

In [26]:
def triple_maker_att_WheelchairAccessible():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('WheelchairAccessible'):
                    if (data_business[n]['attributes']['WheelchairAccessible']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.accessibilityFeature, URIRef(db_resource + 'Wheelchair')))

In [27]:
def triple_maker_att_HappyHour():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('HappyHour'):
                    if (data_business[n]['attributes']['HappyHour']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Happy_Hour')))

In [28]:
def triple_maker_att_OutdoorSeating():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('OutdoorSeating'):
                    if (data_business[n]['attributes']['OutdoorSeating']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.containsPlace, URIRef(db_resource + 'Outdoor_dining')))

In [29]:
def triple_maker_att_HasTV():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('HasTV'):
                    if (data_business[n]['attributes']['HasTV']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Television')))

In [30]:
def triple_maker_att_RestaurantsReservations():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsReservations'):
                    if (data_business[n]['attributes']['RestaurantsReservations']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.acceptsReservations, URIRef(db_resource + 'RestaurantsReservations')))

In [32]:
def triple_maker_att_DogsAllowed():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('DogsAllowed'):
                    if (data_business[n]['attributes']['DogsAllowed']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.petsAllowed, Literal('True', datatype=XSD.boolean)))
                    if (data_business[n]['attributes']['DogsAllowed']) == 'False':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.petsAllowed, Literal('False', datatype=XSD.boolean)))

In [33]:
def triple_maker_att_Alcohol():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Alcohol'):
                    if (data_business[n]['attributes']['Alcohol']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.hasMenuItem, URIRef(db_resource + 'Alcoholic_drink')))

In [34]:
def triple_maker_att_GoodForKids():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('GoodForKids'):
                    if (data_business[n]['attributes']['GoodForKids']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Family-friendly')))

In [35]:
def triple_maker_att_ambience():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Ambience'):
                    if (data_business[n]['attributes']['Ambience']) == "'romantic': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Romantic', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'romantic': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Romantic', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'intimate': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Intimate', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'intimate': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Intimate', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'touristy': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Touristy', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'touristy': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Touristy', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'hipster': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Hipster', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'hipster': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Hipster', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'divey': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Divey', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'divey': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Divey', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'trendy': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Trendy', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'trendy': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Trendy', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'upscale': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Upscale', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'upscale': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Upscale', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'classy': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Classy', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'classy': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Classy', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "'casual': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Casual', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['Ambience']) == "u'casual': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_ambience, Literal('Casual', datatype=XSD.string)))

In [36]:
def triple_maker_att_RestaurantsTableService():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('RestaurantsTableService'):
                    if (data_business[n]['attributes']['RestaurantsTableService']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Waiting_staff')))

In [37]:
def triple_maker_att_DriveThru():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('DriveThru'):
                    if (data_business[n]['attributes']['DriveThru']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Drive-through')))

In [51]:
def triple_maker_att_NoiseLevel():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('NoiseLevel'):
                    if (data_business[n]['attributes']['NoiseLevel']) == "'average'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Average', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['NoiseLevel']) == "u'average'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Average', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['NoiseLevel']) == "'quiet'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Quiet', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['NoiseLevel']) == "u'quiet'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Quiet', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['NoiseLevel']) == "'loud'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Loud', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['NoiseLevel']) == "u'loud'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Loud', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['NoiseLevel']) == "u'very_loud'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Very_loud', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['NoiseLevel']) == "u'very_loud'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.Level, Literal('Very_loud', datatype=XSD.string)))

In [74]:
def triple_maker_att_GoodForMeal():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('GoodForMeal'):
                    if (data_business[n]['attributes']['GoodForMeal']) == "'dessert': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Dessert', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "'dessert': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Dessert', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "'brunch': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Brunch', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "u'brunch': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Brunch', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "'breakfast': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Breakfast', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "u'breakfast': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Breakfast', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "'lunch': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Lunch', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "u'lunch': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Lunch', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "'dinner': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Dinner', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "u'dinner': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Dinner', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "'latenight': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Latenight', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['GoodForMeal']) == "u'latenight': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_goodformeal, Literal('Latenight', datatype=XSD.string)))                    

In [39]:
def triple_maker_att_BusinessAcceptsBitcoin():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BusinessAcceptsBitcoin'):
                    if (data_business[n]['attributes']['BusinessAcceptsBitcoin']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.paymentAccepted, Literal('Cryptocurrency', datatype=XSD.string)))

In [40]:
def triple_maker_att_Music():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('Music'):
                    if (data_business[n]['attributes']['Music']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Music')))

In [41]:
def triple_maker_att_GoodForDancing():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('GoodForDancing'):
                    if (data_business[n]['attributes']['GoodForDancing']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'Dance')))

In [42]:
def triple_maker_att_AcceptsInsurance():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('AcceptsInsurance'):
                    if (data_business[n]['attributes']['AcceptsInsurance']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.paymentAccepted, Literal('Insurance', datatype=XSD.string)))

In [ ]:
def triple_maker_att_BestNights():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BestNights'):
                    if (data_business[n]['attributes']['BestNights']) == "'monday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Monday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "u'monday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Monday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "'tuesday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Tuesday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "u'tuesday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Tuesday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "'wednesday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Wednesday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "u'wednesday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Wednesday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "'thursday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Thursday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "u'thursday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Thursday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "'friday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Friday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "u'friday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Friday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "'saturday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Saturday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "u'saturday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Saturday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "'sunday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Sunday', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['BestNights']) == "u'sunday': False":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), yelp_bestnights, Literal('Sunday', datatype=XSD.string)))

In [43]:
def triple_maker_att_BYOB():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BYOB'):
                    if (data_business[n]['attributes']['BYOB']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))

In [44]:
def triple_maker_att_corkage():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('corkage'):
                    if (data_business[n]['attributes']['corkage']) == 'True':
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))

In [45]:
def triple_maker_att_BYOBCorkage():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('BYOBCorkage'):
                    if (data_business[n]['attributes']['BYOBCorkage']) == "'yes_free'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))
                    if (data_business[n]['attributes']['BYOBCorkage']) == "'yes_corkage'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))
                    if (data_business[n]['attributes']['BYOBCorkage']) == "u'yes_free'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))
                    if (data_business[n]['attributes']['BYOBCorkage']) == "u'yes_corkage'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.amenityFeature, URIRef(db_resource + 'BYOB')))

In [46]:
def triple_maker_att_AgesAllowed():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('AgesAllowed'):
                    if (data_business[n]['attributes']['AgesAllowed']) == "u'18plus'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.suggestedAge, Literal('18+')))
                    if (data_business[n]['attributes']['AgesAllowed']) == "u'21plus'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.suggestedAge, Literal('21+')))
                    if (data_business[n]['attributes']['AgesAllowed']) == "u'allages'":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.suggestedAge, Literal('All_Ages')))

In [83]:
def triple_maker_att_DietaryRestrictions():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            for element in (data_business[n]['attributes']):
                if data_business[n]['attributes'].__contains__('DietaryRestrictions'):
                    if (data_business[n]['attributes']['DietaryRestrictions']) == "'dairy-free': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.RestrictedDiet, Literal('Dairy-free', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['DietaryRestrictions']) == "'gluten-free': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.RestrictedDiet, Literal('Gluten-free', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['DietaryRestrictions']) == "'vegan': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.RestrictedDiet, Literal('Vegan', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['DietaryRestrictions']) == "'kosher': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.RestrictedDiet, Literal('Kosher', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['DietaryRestrictions']) == "'halal': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.RestrictedDiet, Literal('Halal', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['DietaryRestrictions']) == "'soy-free': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.RestrictedDiet, Literal('Soy-free', datatype=XSD.string)))
                    elif (data_business[n]['attributes']['DietaryRestrictions']) == "'vegetarian': True":
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.RestrictedDiet, Literal('Vegetarian', datatype=XSD.string)))

In [79]:
print(data_business[19193]['attributes']['DietaryRestrictions'])

{'dairy-free': False, 'gluten-free': False, 'vegan': False, 'kosher': False, 'halal': False, 'soy-free': False, 'vegetarian': True}


In [58]:
test_list = []
n = 0
for element in data_business:
    if data_business[n]['attributes'] != None:
        if data_business[n]['attributes'].__contains__('BestNights'):
            if data_business[n]['attributes']['BestNights'] not in test_list:
                test_list.append(data_business[n]['attributes']['BestNights'])
    n = n + 1
print(test_list)

In [76]:
test_list = []
n = 0
for element in data_business:
    if data_business[n]['attributes'] != None:
        if data_business[n]['attributes'].__contains__('DietaryRestrictions'):
            s_v2_list = string_seperate_v2(data_business[n]['attributes']['DietaryRestrictions'])
            x = 0
            for words in s_v2_list:
                if s_v2_list[x] not in test_list:
                    test_list.append(s_v2_list[x])
                x = x + 1
    n = n + 1

print(test_list)

["'dairy-free': False", "'gluten-free': False", "'vegan': True", "'kosher': False", "'halal': False", "'soy-free': False", "'vegetarian': True", "'gluten-free': True", "'vegan': False", "'vegetarian': False", "'dairy-free': True", "'soy-free': True", 'None', "'halal': True"]


In [ ]:
lol = 10

print(type(data_business[lol]['attributes']))
print(data_business[lol]['attributes'])
print(type(string_seperate(data_business[lol]['attributes']['BusinessParking'])))
print(string_seperate(data_business[lol]['attributes']['BusinessParking']))

In [ ]:
'''
triple_maker_name()
triple_maker_address()
triple_maker_state()
triple_maker_postal()
triple_maker_latlong()
triple_maker_stars()
triple_maker_cg_cuisine()
triple_maker_cg_in_db_exist()
'''

In [ ]:
#g.serialize(destination="Business_v2.ttl")

In [ ]:
#triple_maker_cg_in_db_exist_2()

#print(category_exist)
#print(category_not_exist)